In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
import requests
from bs4 import BeautifulSoup
import re

# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://namu.wiki/RecentChanges"

# 사이트의 html 구조에 기반하여 크롤링을 수행합니다
req = requests.get(source_url)
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name="table")
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

# 페이지 리스트 가져오기

In [3]:
# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all('td')[0]
    td_url = first_td.find_all('a')
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get('href')
        if 'png' not in page_url:
            page_urls.append(page_url)
            
# 중복 url을 제거합니다
page_urls = list(set(page_urls))
for page in page_urls[:5]:
    print(page)

https://namu.wiki/w/%EC%95%BC%EC%95%A0%EB%8B%88%20%EC%B1%84%EB%84%90
https://namu.wiki/w/%EB%A7%88%EB%B2%95%20%EC%86%8C%EB%85%80(day7)
https://namu.wiki/w/%EC%9C%A0%EC%96%B8/%ED%95%9C%EA%B5%AD%20%EC%B0%BD%EC%9E%91%EB%AC%BC
https://namu.wiki/w/%EB%8D%94%EB%AF%B8:Amc82
https://namu.wiki/w/%EC%BC%80%EB%B9%88%20%EB%8D%94%EB%B8%8C%EB%9D%BC%EC%9C%84%EB%84%88


# 페이지 내 텍스트 구조 확인

In [4]:
req = requests.get(page_urls[0])
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name = 'article')
title = contents_table.find_all('h1')[0]
category = contents_table.find_all('ul')[0]
content_paragraphs = contents_table.find_all(name="div", attrs={"class" : "wiki-paragraph"})
content_corpus_list = []

for paragraphs in content_paragraphs:
    content_corpus_list.append(paragraphs.text)
content_corpus = "".join(content_corpus_list)

print(title.text)
print("\n")
print(category.text)
print("\n")
print(content_corpus)

IndexError: list index out of range

# 나무위키 최근 변경 데이터 크롤링

In [ ]:
columns = ['title', 'category', 'content_text']
df = pd.DataFrame(columns=columns)

# 각 페이지별 '제목', '카테고리', '본문' 정보를 데이터 프레임으로 만듭니다.
for page_url in page_urls:

    # 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
    req = requests.get(page_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    contents_table = soup.find(name="article")
    title = contents_table.find_all('h1')[0]
    
    # 카테고리 정보가 없는 경우를 확인합니다.
    if len(contents_table.find_all('ul')) > 0:
        category = contents_table.find_all('ul')[0]
    else:
        category = None
    
    content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
    content_corpus_list = []
        
        
    # 페이지 내 제목 정보에서 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""
        
    # 페이지 내 본문 정보에서 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우 빈 문자열로 대체합니다.
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n", " "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")
        
    # 페이지 내 카테고리정보에서 "분류"라는 단어와 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우 빈 문자열로 대체합니다.
    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""

    # 모든 정보를 하나의 데이터프레임에 저장합니다
    row = [row_title, row_category, "".join(content_corpus_list)]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)

In [ ]:
df.head()

# 키워드 추출
## [텍스트 데이터 전처리]

In [ ]:
# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거합니다
def text_cleaning(text):
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+') #  한글의 정규표현식을 나타냅니다
    result = hangul.sub('', text)
    return result

In [ ]:
print(text_cleaning(df['title'][0]))

## 말뭉치 만들기

In [ ]:
# 각 피처마다 말뭉치를 생성합니다.
title_corpus = "".join(df['title'].tolist())
category_corpus = "".join(df['category'].tolist())
content_corpus = "".join(df['content_text'].tolist())

title_corpus

In [ ]:
!pip install JPype1‑1.2.0‑cp38‑none‑win_amd64.whl

In [5]:
from konlpy.tag import Okt
from collections import Counter

# konlpy의 형태소 분석기로 명사 단위의 키워드를 추출합니다
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(content_corpus)
count = Counter(nouns)
count

SystemError: java.nio.file.InvalidPathException: Illegal char <*> at index 61: C:\Users\d\anaconda3\envs\test\lib\site-packages\konlpy\java\*